In [ ]:

def collect_data_p(p: str, datasets=[], output_dir='results', test=True):
    """
    Collects data from multiple datasets and methods for a specific missingness percentage,
    and formats it into a DataFrame.
    
    Args:
        p (str): The missingness percentage (e.g., "0.0").
        datasets (list): List of dataset names to analyze.
        output_dir (str): The base directory where results are saved.
        
    Returns:
        pd.DataFrame: A DataFrame containing formatted results for each method and imputation.
    """
    import pandas as pd
    import os
    import json
    import numpy as np
    import re
    
    if not datasets:
        # If no datasets are specified, get all available datasets from the directory
        datasets = [d for d in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir, d))]
    
    # Initialize an empty list to store data rows
    data_rows = []
    
    # First pass: collect all methods and imputations across all datasets
    all_method_imputation_pairs = []
    
    for dataset in datasets:
        dataset_dir = os.path.join(output_dir, dataset)
        if not os.path.exists(dataset_dir):
            print(f"Warning: Dataset directory {dataset_dir} not found. Skipping.")
            continue
            
        # Get all method directories for this dataset
        method_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
        
        for method in method_dirs:
            method_dir = os.path.join(dataset_dir, method)
            
            for imputation in os.listdir(method_dir):
                
            
                method_imputation_pair = (method, imputation)
                if method_imputation_pair not in all_method_imputation_pairs:
                    all_method_imputation_pairs.append(method_imputation_pair)
    # print(all_method_imputation_pairs)

    # Second pass: collect results for each method/imputation pair across all datasets
    for method, imputation in all_method_imputation_pairs:
        row = {
            'method': method,
            'imputation': imputation
        }
        
        # Collect results for each dataset
        for dataset in datasets:
            dataset_dir = os.path.join(output_dir, dataset)
            if not os.path.exists(dataset_dir):
                print(f"{method_dir} not exits!")
                # Dataset doesn't exist, fill with N/A
                row[f"{dataset}_PEHE"] = "N/A"
                row[f"{dataset}_MAE"] = "N/A"
                continue
                
            method_dir = os.path.join(dataset_dir, method)
            if not os.path.exists(method_dir):
                # Method doesn't exist for this dataset, fill with N/A
                row[f"{dataset}_PEHE"] = "N/A"
                row[f"{dataset}_MAE"] = "N/A"
                continue
            
            imputation_dir = os.path.join(method_dir, imputation)
            train_test_flag = "test" if test else "train"
            # Get all result files for this method with the specified p value
            results_files = [f for f in os.listdir(imputation_dir) 
                        if f.startswith(f"p={p}_") and f.endswith(f"_{train_test_flag}_results.json")]
            
            # print(results_files)
            effect_pehe_values = []
            effect_mae_values = []
            
            # Collect metrics from all matching files
            for file in results_files:
                file_path = os.path.join(method_dir, imputation, file)
                try:
                    with open(file_path, 'r') as f:
                        results = json.load(f)
                        if 'effect_pehe' in results:
                            effect_pehe_values.append(results['effect_pehe'])
                        if 'effect_mae' in results:
                            effect_mae_values.append(results['effect_mae'])
                except (json.JSONDecodeError, FileNotFoundError) as e:
                    print(f"Warning: Error reading file {file_path}: {e}")
            # print("PHHE", effect_pehe_values)
            # print("MAE", effect_mae_values)
            # Format mean and std for each metric
            if effect_pehe_values:
                mean_pehe = np.mean(effect_pehe_values)
                std_pehe = np.std(effect_pehe_values)
                row[f"{dataset}_PEHE"] = f"{mean_pehe:.2f} ± {std_pehe:.2f}"
            else:
                row[f"{dataset}_PEHE"] = "N/A"
            
            if effect_mae_values:
                mean_mae = np.mean(effect_mae_values)
                std_mae = np.std(effect_mae_values)
                row[f"{dataset}_MAE"] = f"{mean_mae:.2f} ± {std_mae:.2f}"
            else:
                row[f"{dataset}_MAE"] = "N/A"
        # print(row)
        data_rows.append(row)
    
    # Create DataFrame
    df = pd.DataFrame(data_rows)
    # print(df)
    
    # Sort the DataFrame by method and imputation
    if not df.empty:
        df = df.sort_values(by=['method', 'imputation'])
    
    # # Reorder columns to match the required order
    # if not df.empty:
    #     columns = ['method', 'imputation']
    #     for dataset in datasets:
    #         columns.extend([f"{dataset}_PEHE", f"{dataset}_AME"])
        
    #     # Make sure all columns exist before reordering
    #     existing_columns = [col for col in columns if col in df.columns]
    #     df = df[existing_columns]
    
    return df

def display_results(p, datasets=["Syn", "Youtube"]):
    import pandas as pd

    # Set display options to show all rows and columns
    pd.set_option('display.max_rows', None)  # Show all rows
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.width', None)  # Use the full width of the notebook cell
    pd.set_option('display.max_colwidth', None)  # Show full content of each column
    
    # Example: Display the DataFrame created by collect_data_p
    df = collect_data_p(p, datasets=datasets) #, "AMZS", "Flickr"])
    
    # Display the DataFrame
    display(df)

In [3]:
display_results(p='0.0')
display_results(p='0.1')
display_results(p='0.3')
display_results(p='0.5')


""


""


""


""
